In [1]:
from imports import *
from functions import *
from constants import *

In [2]:
try:
    driver = create_driver()
    # Suppression de l'élément Cambiar Pais
    driver.execute_script("""
    var element = document.querySelector("#cambiarPais");
    if (element) {
        element.remove();  // Cette ligne supprime l'élément du DOM
    }
""")
    driver.refresh()
    wait = WebDriverWait(driver, 10)
except :
    pass

# Connection to Account

In [3]:

try:
    driver.get(website_url)

    # Find Mi Cuenta and click on it
    mi_cuenta = wait.until(EC.element_to_be_clickable((By.XPATH,
        "//a[contains(@title, 'Mi Cuenta')]")))
    if not mi_cuenta.is_selected():
        mi_cuenta.click()
       
except Exception as e:
    print(str(e))

try:
    email_input = wait.until(EC.presence_of_element_located((By.XPATH, 
                    "//input[contains(@id, 'signin_username')]")))
    if email_input:
        print("Email Input found")
        email_input.send_keys(my_email)
        email_input.send_keys(Keys.ENTER)

except Exception as e:
    print(str(e)) 

try:
    password_input = wait.until(EC.presence_of_element_located((By.XPATH, 
                    "//input[contains(@id, 'signin_password')]")))
    if password_input:
        print("Password Input found")
        password_input.send_keys(my_password)
        password_input.send_keys(Keys.ENTER)

except Exception as e:
    print(str(e)) 


Email Input found
Password Input found


# Queries

In [7]:
QUERY_CATEGORY = "technology"
QUERY_SEARCH = ""
QUERY_BOOK_STATE = "new"
QUERY_BOOK_FORMAT = "paper"
place_category_menu = wait.until(EC.presence_of_all_elements_located((By.XPATH, 
                        "//li[contains(@class, 'category-li') and contains(@data-category-id,'')]")))
place_products_listings = find_element(driver, "xpath", "//div[contains(@class, 'productos')]")
place_cards = wait.until(EC.presence_of_all_elements_located((By.XPATH, 
                "//div[contains(@class, 'box-producto') and contains(@data-id_producto, '')]")))

In [ ]:
# try:
#     Categories_menu = wait.until(EC.presence_of_element_located((By.XPATH, 
#                         "//div[contains(@class, 'categories col-left')]")))
#     print('Category menu Found !')
# except Exception as e:
#     print(f"❌ Error: {str(e)}")

try:
    categories = place_category_menu
    print('Categories Found !')

    if QUERY_CATEGORY:
        query_category = process_string(QUERY_CATEGORY).strip()  # Traite la chaîne de recherche
        print(f"🔍 Query: {query_category}")


    for cat in categories:
        category_name =cat.text.strip()
        link = find_element(cat, "xpath", ".//a[contains(@itemprop, 'WPSideBar')]")
        category_href = link.get_attribute("href")
        print(f"🔄 Comparing:\n'{query_category}'\n'{category_name}'")
        print(f"🔗 Category link: {category_href}\n")
        # Si une correspondance est trouvée# Si une correspondance est trouvée
        if query_category in category_name.lower():
            print(f"✅ Match found! Navigating to: {category_href}")
            driver.get(category_href)
            break  # Arrêter dès qu'on trouve la correspondance

except Exception as e:
    print(f"❌ Error: {str(e)}")

Filters Choice

In [8]:
if all(x in QUERY_BOOK_STATE for x in ["new", "used"]):
    current_url = driver.current_url.split("?")[0]
    url_new_and_used_books =  current_url + "?condition=nuevosyusados"
    driver.get(url_new_and_used_books)

elif any("new" in q.lower() for q in QUERY_BOOK_STATE) :
    current_url = driver.current_url.split("?")[0]
    url_new_books =  current_url + "?condition=new"
    driver.get(url_new_books)

elif any("used" in q.lower() for q in QUERY_BOOK_STATE):
    current_url = driver.current_url.split("?")[0]
    url_new_books =  current_url + "?condition=used"
    driver.get(url_new_books)
else:
    pass

if QUERY_BOOK_STATE.lower() == "ebook":
    current_url = driver.current_url.split("?")[0]
    url_new_and_used_books =  current_url + "?condition=ebook"
    driver.get(url_new_and_used_books)

elif "audio" in QUERY_BOOK_STATE.lower():
    current_url = driver.current_url.split("?")[0]
    url_new_and_used_books =  current_url + "?condition=audio"
    driver.get(url_new_and_used_books)
else:
    pass

One Card

In [ ]:
Products_listings = place_products_listings

cards = place_cards
card =cards[0]

Products_listings = place_products_listings
cards = place_cards

title = card.find_element(By.XPATH, ".//h3[contains(@class, 'nombre')]").text.strip()  
print(f"Title: {title}")

href = card.find_element(By.XPATH, ".//a[contains(@href, '')]").get_attribute('href')
print(f"Link: {href}")

autor = find_element(card, "xpath", 
    ".//div[contains(@class, 'autor')]").text.strip()
print(f"Author: {autor}")
edition = find_element(card, "xpath", 
    ".//div[contains(@class, 'autor') and contains(@class,'color-dark-gray')]").text.strip()
print(f"Edition: {edition}")
stock = find_element(card, "xpath", ".//div[contains(@class, 'stock') and contains(@class,'green')]").text.strip()
print(f"Stock status: {stock}")

price = find_element(card, "xpath", 
                            ".//p[contains(@class, 'precio-ahora')]").text.strip()
print(f"Price: {price}")
precio_antes = find_element(card, "xpath", 
                            ".//p[contains(@class, 'precio-antes')]").text.strip()
print(f"Old Price: {precio_antes}\n")
match_digit_precio = re.search(r'\d+\.\d+', price)
match_digit_precio_antes = re.search(r'\d+\.\d+', precio_antes)

discount = f"{float(match_digit_precio)/float(match_digit_precio_antes)}%"
print(f"Discount: {discount}")
driver.get(href)

Title: La ola que Viene
Link: https://www.buscalibre.com.co/libro-la-ola-que-viene/9788419399649/p/55586326
Author: Suleyman, Mustafa ; Bhaskar, Michael
Edition: Debate, 2023, 1 Edición, Tapa Blanda, Nuevo
Stock status: Quedan 23 unidades
Price: $ 97.386
Old Price: $ 194.772



ValueError: could not convert string to float: '$ 97.386'

Collect Datas

In [97]:
Products_listings = place_products_listings

if Products_listings:
    cards = place_cards
    if cards:
        print("Cards Found !")
        for card in cards:
            Products_listings = place_products_listings
            cards = place_cards

            title = card.find_element(By.XPATH, ".//h3[contains(@class, 'nombre')]").text.strip()  
            print(f"Title: {title}")

            href = card.find_element(By.XPATH, ".//a[contains(@href, '')]").get_attribute('href')
            print(f"Link: {href}")

            autor = find_element(card, "xpath", 
                ".//div[contains(@class, 'autor')]").text.strip()
            print(f"Author: {autor}")
            edition = find_element(card, "xpath", 
                ".//div[contains(@class, 'autor') and contains(@class,'color-dark-gray')]").text.strip()
            print(f"Edition: {edition}")
            stock = find_element(card, "xpath", ".//div[contains(@class, 'stock') and contains(@class,'green')]").text.strip()
            print(f"Stock status: {stock}")
            
            price = find_element(card, "xpath", 
                                        ".//p[contains(@class, 'precio-ahora')]").text.strip()
            print(f"Price: {price}")
            precio_antes = find_element(card, "xpath", 
                                        ".//p[contains(@class, 'precio-antes')]").text.strip()
            print(f"Old Price: {precio_antes}\n")

Cards Found !
Title: La ola que Viene
Link: https://www.buscalibre.com.co/libro-la-ola-que-viene/9788419399649/p/55586326
Author: Suleyman, Mustafa ; Bhaskar, Michael
Edition: Debate, 2023, 1 Edición, Tapa Blanda, Nuevo
Stock status: Quedan 25 unidades
Price: $ 97.386
Old Price: $ 194.772

Title: Videojuegos
Link: https://www.buscalibre.com.co/libro-videojuegos/9788496924246/p/1501329
Author: Vincent Montagnana
Edition: Ma Non Troppo, 2008, 1 Edición, Tapa Blanda, Usado
Stock status: Queda 1 unidad
Price: $ 64.454
Old Price: $ 128.908

Title: Infocracia
Link: https://www.buscalibre.com.co/libro-infocracia/9789585165212/p/54072616
Author: Byung-Chul Han
Edition: Taurus, 2022, 1 Edición, Tapa Blanda, Nuevo
Stock status: Quedan 29 unidades
Price: $ 31.500
Old Price: $ 45.000

Title: El Pequeño Libro Para Invertir con Sentido Común: El Mejor Método Para Garantizar la Rentabilidad en Bolsa
Link: https://www.buscalibre.com.co/libro-el-pequeno-libro-para-invertir-con-sentido-comun-el-mejor-me